In [12]:
"""
filename: estimize_cons_eadates

Get expected EA dates from their poorly-named data column with dates, reports_on. 
Then joins together using whatever's available in their get releases API to get 
actual EA dates. 

Inputs: raw data CSV files sent over privately in a URL on 4/12/2022
These are 
-combined_estimates_new.csv
-combined_consensus_new.csv

This particular EA processing file uses combined_consensus_new.csv. 

Output: estimize_eadates.parquet

"""
# Imports here 
import pandas as pd 
from datetime import date, timedelta, datetime
from dateutil import relativedelta
import time
from optparse import OptionParser
import numpy as np
import sqlite3
import pytz
import requests

# Paths defined here
onedrive = "C://Users/clj585/OneDrive - Northwestern University/"
out_pth = "data_feeds/estimize/data/"
raw_pth = onedrive + "data_feeds/estimize/data/Estimize_CSVS/"

# Load CSVs here
cons = pd.read_csv(raw_pth + "combined_consensus_new.csv")
cons

C:\Users\clj585\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (2,26) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,date,ticker,cusip,instrument_id,instrument_name,instrument_sector,instrument_industry,fiscal_year,fiscal_quarter,reports_at,...,estimize.revenue.low,estimize.revenue.sd,estimize.revenue.count,wallstreet.eps,wallstreet.revenue,actual.eps,actual.revenue,release_id,point_in_time_ticker,point_in_time_cusip
0,2013-01-31,MLM,573284106,1,Martin Marietta Materials Inc.,Materials,Construction Materials,2012,4,2013-02-12T06:00:00-05:00,...,422.90,24.763149,4.0,NaN,NaN,0.46,504.100,170,MLM,573284106
1,2013-02-01,MLM,573284106,1,Martin Marietta Materials Inc.,Materials,Construction Materials,2012,4,2013-02-12T06:00:00-05:00,...,422.90,24.763149,4.0,NaN,NaN,0.46,504.100,170,MLM,573284106
2,2013-02-02,MLM,573284106,1,Martin Marietta Materials Inc.,Materials,Construction Materials,2012,4,2013-02-12T06:00:00-05:00,...,422.90,24.763149,4.0,NaN,NaN,0.46,504.100,170,MLM,573284106
3,2013-02-03,MLM,573284106,1,Martin Marietta Materials Inc.,Materials,Construction Materials,2012,4,2013-02-12T06:00:00-05:00,...,422.90,22.185673,5.0,NaN,NaN,0.46,504.100,170,MLM,573284106
4,2013-02-04,MLM,573284106,1,Martin Marietta Materials Inc.,Materials,Construction Materials,2012,4,2013-02-12T06:00:00-05:00,...,422.90,19.102785,7.0,NaN,NaN,0.46,504.100,170,MLM,573284106
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6973355,2022-02-26,CRON,22717L101,3337,Cronos Group Inc.,Health Care,Pharmaceuticals,2021,4,2022-03-01T07:48:03-05:00,...,23.68,2.471023,3.0,-0.1050,23.901000,-0.36,25.795,210178,CRON,22717L101
6973356,2022-02-27,CRON,22717L101,3337,Cronos Group Inc.,Health Care,Pharmaceuticals,2021,4,2022-03-01T07:48:03-05:00,...,23.68,2.471023,3.0,-0.1050,23.901000,-0.36,25.795,210178,CRON,22717L101
6973357,2022-02-28,CRON,22717L101,3337,Cronos Group Inc.,Health Care,Pharmaceuticals,2021,4,2022-03-01T07:48:03-05:00,...,23.68,2.471023,3.0,-0.1050,23.901000,-0.36,25.795,210178,CRON,22717L101
6973358,2022-03-01,CRON,22717L101,3337,Cronos Group Inc.,Health Care,Pharmaceuticals,2021,4,2022-03-01T07:48:03-05:00,...,23.68,2.471023,3.0,-0.1050,23.901000,-0.36,25.795,210178,CRON,22717L101


In [13]:
"""
From consensus data, select columns and format as strings initially. 
Rename columns according to our standard scheme. 

"""

collist = ['date', 'ticker', 'cusip', "instrument_id", 'fiscal_quarter', "fiscal_year", 
           "reports_at", "estimize.eps.weighted", "wallstreet.eps", "actual.eps", "release_id",
           "point_in_time_ticker", "point_in_time_cusip", "estimize.revenue.weighted", 
          "wallstreet.revenue", "actual.revenue"]

# Column select
cons2 = cons[collist]

# Note: the CUSIP columns here have 9 alphanumeric characters! 
colmap = {"reports_at" : "exp_ea_datetime", "instrument_id" : "estimize_id", "ticker" : "cticker", 
            "cusip" : "ccusip9", "point_in_time_ticker" : "hticker", "point_in_time_cusip" : "hcusip9", 
         "fiscal_year" : "fyear", "fiscal_quarter" : "fqtr", "wallstreet.eps":"eps_ws", 
         "actual.eps" : "eps_act", "actual.revenue" : "sales_act", "estimize.revenue.weighted":"sales_wgt", 
         "wallstreet.revenue":"sales_ws", "estimize.eps.weighted" :"eps_wgt"}
cons2 = cons2.rename(columns = colmap)
cons2[['fyear', "fqtr", "estimize_id"]] = cons2[['fyear', "fqtr",
                                                 "estimize_id"]].astype("Int64")
cons2[['exp_ea_datetime', "cticker", 
       "ccusip9", "hticker", "hcusip9"]] = cons2[['exp_ea_datetime', "cticker", "ccusip9",
                                                 "hticker", "hcusip9"]].astype("string")
cons2

,date,cticker,ccusip9,estimize_id,fqtr,fyear,exp_ea_datetime,eps_wgt,eps_ws,eps_act,release_id,hticker,hcusip9,sales_wgt,sales_ws,sales_act
0,2013-01-31,MLM,573284106,1,4,2012,2013-02-12T06:00:00-05:00,0.452788,NaN,0.46,170,MLM,573284106,440.402364,NaN,504.100
1,2013-02-01,MLM,573284106,1,4,2012,2013-02-12T06:00:00-05:00,0.452733,NaN,0.46,170,MLM,573284106,440.556339,NaN,504.100
2,2013-02-02,MLM,573284106,1,4,2012,2013-02-12T06:00:00-05:00,0.452733,NaN,0.46,170,MLM,573284106,440.556339,NaN,504.100
3,2013-02-03,MLM,573284106,1,4,2012,2013-02-12T06:00:00-05:00,0.469858,NaN,0.46,170,MLM,573284106,443.003012,NaN,504.100
4,2013-02-04,MLM,573284106,1,4,2012,2013-02-12T06:00:00-05:00,0.474410,NaN,0.46,170,MLM,573284106,441.871531,NaN,504.100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6973355,2022-02-26,CRON,22717L101,3337,4,2021,2022-03-01T07:48:03-05:00,-0.024769,-0.1050,-0.36,210178,CRON,22717L101,24.668085,23.901000,25.795
6973356,2022-02-27,CRON,22717L101,3337,4,2021,2022-03-01T07:48:03-05:00,-0.024769,-0.1050,-0.36,210178,CRON,22717L101,24.668085,23.901000,25.795
6973357,2022-02-28,CRON,22717L101,3337,4,2021,2022-03-01T07:48:03-05:00,-0.024769,-0.1050,-0.36,210178,CRON,22717L101,24.668085,23.901000,25.795
6973358,2022-03-01,CRON,22717L101,3337,4,2021,2022-03-01T07:48:03-05:00,-0.024769,-0.1050,-0.36,210178,CRON,22717L101,24.668085,23.901000,25.795


In [14]:
"""
Ticker symbology - processing their format, which is the share 
classes separated by a dash. There appear to be no prefixes. 

"""
# Split and refill with empty string columns 
cons2[["root", 'suffix']] = cons2['hticker'].str.split("-", n=1, expand = True)
cons2['suffix'] = cons2['suffix'].fillna("")

# Adjust for the fact that there is defunct first in some htickers! 
cons2['root'] = np.where(cons2['hticker'].str.contains("DEFUNCT"), cons2['suffix'], cons2['root'])
cons2['suffix'] = np.where(cons2['hticker'].str.contains("DEFUNCT"), "", cons2['suffix'])

# This is for cosmetic purposes - can't have that DEFUNCT string in hticker
cons2['hticker'] = cons2['hticker'].replace('DEFUNCT - ', '', regex=True)

# Checking once with Berkshire B class shares
cons2[cons2['hticker']=="BRK-B"]

,date,cticker,ccusip9,estimize_id,fqtr,fyear,exp_ea_datetime,eps_wgt,eps_ws,eps_act,release_id,hticker,hcusip9,sales_wgt,sales_ws,sales_act,root,suffix
2364545,2014-07-23,BRK-B,084670702,998,2,2014,2014-08-01T12:00:00-04:00,1.640761,NaN,1.76,52199,BRK-B,084670702,48047.274110,NaN,49760.0,BRK,B
2364546,2014-07-24,BRK-B,084670702,998,2,2014,2014-08-01T12:00:00-04:00,1.640775,NaN,1.76,52199,BRK-B,084670702,48045.816898,NaN,49760.0,BRK,B
2364547,2014-07-25,BRK-B,084670702,998,2,2014,2014-08-01T12:00:00-04:00,1.640775,NaN,1.76,52199,BRK-B,084670702,48045.816898,NaN,49760.0,BRK,B
2364548,2014-07-26,BRK-B,084670702,998,2,2014,2014-08-01T12:00:00-04:00,1.640775,NaN,1.76,52199,BRK-B,084670702,48045.816898,NaN,49760.0,BRK,B
2364549,2014-07-27,BRK-B,084670702,998,2,2014,2014-08-01T12:00:00-04:00,1.640775,NaN,1.76,52199,BRK-B,084670702,48045.816898,NaN,49760.0,BRK,B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2368365,2022-03-27,BRK-B,084670702,998,1,2022,2022-05-02T06:00:00-04:00,3.128762,2.957552,NaN,189062,BRK-B,084670702,70549.509727,70189.822513,NaN,BRK,B
2368366,2022-03-28,BRK-B,084670702,998,1,2022,2022-05-02T06:00:00-04:00,3.128762,2.957552,NaN,189062,BRK-B,084670702,70549.509727,70189.822513,NaN,BRK,B
2368367,2022-03-29,BRK-B,084670702,998,1,2022,2022-05-02T06:00:00-04:00,3.128762,2.957552,NaN,189062,BRK-B,084670702,70549.509727,70189.822513,NaN,BRK,B
2368368,2022-03-30,BRK-B,084670702,998,1,2022,2022-05-02T06:00:00-04:00,3.128762,2.957552,NaN,189062,BRK-B,084670702,70549.509727,70189.822513,NaN,BRK,B


In [15]:
"""
One test case found here with DEFUNCT being out of place 

"""
print(len(cons2[cons2['hticker'].str.contains("DEFUNCT")]))
cons2[cons2['cticker']=="DEFUNCT - MR"]

7


,date,cticker,ccusip9,estimize_id,fqtr,fyear,exp_ea_datetime,eps_wgt,eps_ws,eps_act,release_id,hticker,hcusip9,sales_wgt,sales_ws,sales_act,root,suffix
3038238,2013-02-25,DEFUNCT - MR,602675100,1333,4,2012,2013-02-25T16:00:00-05:00,0.462158,NaN,0.51,69268,DEFUNCT - MR,602675100,306.438915,NaN,316.100,MR,
3038239,2013-05-06,DEFUNCT - MR,602675100,1333,1,2013,2013-05-06T16:00:00-04:00,0.436024,0.417,0.53,69294,DEFUNCT - MR,602675100,259.210342,256.8,242.100,MR,
3038240,2013-08-05,DEFUNCT - MR,602675100,1333,2,2013,2013-08-05T16:00:00-04:00,0.489311,0.470,0.56,69304,DEFUNCT - MR,602675100,317.133097,319.3,307.200,MR,
3038241,2015-03-06,DEFUNCT - MR,602675100,1333,4,2014,2015-03-09T16:00:00-04:00,0.507456,0.520,0.50,69394,DEFUNCT - MR,602675100,409.593548,417.6,398.955,MR,
3038242,2015-03-07,DEFUNCT - MR,602675100,1333,4,2014,2015-03-09T16:00:00-04:00,0.507456,0.520,0.50,69394,DEFUNCT - MR,602675100,409.593548,417.6,398.955,MR,
3038243,2015-03-08,DEFUNCT - MR,602675100,1333,4,2014,2015-03-09T16:00:00-04:00,0.507456,0.520,0.50,69394,DEFUNCT - MR,602675100,409.593548,417.6,398.955,MR,
3038244,2015-03-09,DEFUNCT - MR,602675100,1333,4,2014,2015-03-09T16:00:00-04:00,0.507426,0.520,0.50,69394,DEFUNCT - MR,602675100,409.593407,417.6,398.955,MR,


In [16]:
"""
More date and time column munging to get EA dates and approximate 
timestamps. 

"""

# Get rid of the T delimiter in the timestamp initially 
cons2['temp'] = cons2['exp_ea_datetime'].str.replace("T", " ", regex=True)

# Filter out the offsset (will be automatically handled and detected by tz_localize)
cons2['temp2'] = cons2['temp'].str[0:19]

# Get the Eastern timestamp from converted to datetime column 
cons2['temp2'] = pd.to_datetime(cons2['temp2'])
cons2['exp_ea_time'] = pd.to_datetime(cons2['exp_ea_datetime'].str[11:19]).dt.time 

# Make a flag column denoting the time of day the announcement happened in trade hours 
cons2['exp_ea_tod'] = np.where(pd.to_datetime(cons2['exp_ea_time'].astype("string"))<"09:30", 
                              "PO", 
                              np.where(pd.to_datetime(cons2['exp_ea_time'].astype("string"))>="16:00", 
                                       "PC", "RH"))

# Last but not least get the expected ea_date extracted
cons2['exp_ea_date'] = cons2['temp2'].dt.strftime('%Y-%m-%d')

cons2 = cons2[['date','cticker','ccusip9','estimize_id','fqtr','fyear',
               'exp_ea_datetime','release_id','hticker', 
               'hcusip9','root','suffix','exp_ea_date','exp_ea_time', "exp_ea_tod",
               'eps_wgt', "eps_act", "eps_ws", "sales_wgt", "sales_act", "sales_ws"
              ]]

# CUSIP munging here? It is erroneous if done at end 
cons2['hcusip'] = cons2['hcusip9'].str[0:8]
cons2['ccusip'] = cons2['ccusip9'].str[0:8]
cons2

<ipython-input-16-1fa2340df4c8>:33: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cons2['hcusip'] = cons2['hcusip9'].str[0:8]


,date,cticker,ccusip9,estimize_id,fqtr,fyear,exp_ea_datetime,release_id,hticker,hcusip9,...,exp_ea_time,exp_ea_tod,eps_wgt,eps_act,eps_ws,sales_wgt,sales_act,sales_ws,hcusip,ccusip
0,2013-01-31,MLM,573284106,1,4,2012,2013-02-12T06:00:00-05:00,170,MLM,573284106,...,06:00:00,PO,0.452788,0.46,NaN,440.402364,504.100,NaN,57328410,57328410
1,2013-02-01,MLM,573284106,1,4,2012,2013-02-12T06:00:00-05:00,170,MLM,573284106,...,06:00:00,PO,0.452733,0.46,NaN,440.556339,504.100,NaN,57328410,57328410
2,2013-02-02,MLM,573284106,1,4,2012,2013-02-12T06:00:00-05:00,170,MLM,573284106,...,06:00:00,PO,0.452733,0.46,NaN,440.556339,504.100,NaN,57328410,57328410
3,2013-02-03,MLM,573284106,1,4,2012,2013-02-12T06:00:00-05:00,170,MLM,573284106,...,06:00:00,PO,0.469858,0.46,NaN,443.003012,504.100,NaN,57328410,57328410
4,2013-02-04,MLM,573284106,1,4,2012,2013-02-12T06:00:00-05:00,170,MLM,573284106,...,06:00:00,PO,0.474410,0.46,NaN,441.871531,504.100,NaN,57328410,57328410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6973355,2022-02-26,CRON,22717L101,3337,4,2021,2022-03-01T07:48:03-05:00,210178,CRON,22717L101,...,07:48:03,PO,-0.024769,-0.36,-0.1050,24.668085,25.795,23.901000,22717L10,22717L10
6973356,2022-02-27,CRON,22717L101,3337,4,2021,2022-03-01T07:48:03-05:00,210178,CRON,22717L101,...,07:48:03,PO,-0.024769,-0.36,-0.1050,24.668085,25.795,23.901000,22717L10,22717L10
6973357,2022-02-28,CRON,22717L101,3337,4,2021,2022-03-01T07:48:03-05:00,210178,CRON,22717L101,...,07:48:03,PO,-0.024769,-0.36,-0.1050,24.668085,25.795,23.901000,22717L10,22717L10
6973358,2022-03-01,CRON,22717L101,3337,4,2021,2022-03-01T07:48:03-05:00,210178,CRON,22717L101,...,07:48:03,PO,-0.024769,-0.36,-0.1050,24.668085,25.795,23.901000,22717L10,22717L10


In [17]:
# This is so annoying 
cons2['cticker'] = np.where(cons2['cticker'].str.contains(" - DEFUNCT"), 
                            cons2['cticker'].str.replace(' ', ''),
                            cons2['cticker'])
cons2[cons2['cticker'].str.contains("DEFUNCT")]

,date,cticker,ccusip9,estimize_id,fqtr,fyear,exp_ea_datetime,release_id,hticker,hcusip9,...,exp_ea_time,exp_ea_tod,eps_wgt,eps_act,eps_ws,sales_wgt,sales_act,sales_ws,hcusip,ccusip
77000,2013-01-26,DOW-DEFUNCT,260543103,40,4,2012,2013-01-31T06:00:00-05:00,2306,DOW,260543103,...,06:00:00,PO,0.352743,0.33,NaN,13658.673355,13917.000,NaN,26054310,26054310
77001,2013-01-27,DOW-DEFUNCT,260543103,40,4,2012,2013-01-31T06:00:00-05:00,2306,DOW,260543103,...,06:00:00,PO,0.352743,0.33,NaN,13658.673355,13917.000,NaN,26054310,26054310
77002,2013-01-28,DOW-DEFUNCT,260543103,40,4,2012,2013-01-31T06:00:00-05:00,2306,DOW,260543103,...,06:00:00,PO,0.352610,0.33,NaN,13657.489397,13917.000,NaN,26054310,26054310
77003,2013-01-29,DOW-DEFUNCT,260543103,40,4,2012,2013-01-31T06:00:00-05:00,2306,DOW,260543103,...,06:00:00,PO,0.356106,0.33,NaN,13682.098109,13917.000,NaN,26054310,26054310
77004,2013-01-30,DOW-DEFUNCT,260543103,40,4,2012,2013-01-31T06:00:00-05:00,2306,DOW,260543103,...,06:00:00,PO,0.329598,0.33,NaN,13639.752535,13917.000,NaN,26054310,26054310
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5891375,2019-07-26,HIVE-DEFUNCT,007786106,2428,2,2019,2019-07-30T18:29:38-04:00,154379,HIVE,007786106,...,18:29:38,PC,-0.006975,0.03,-0.013,37.789374,38.037,37.5,00778610,00778610
5891376,2019-07-27,HIVE-DEFUNCT,007786106,2428,2,2019,2019-07-30T18:29:38-04:00,154379,HIVE,007786106,...,18:29:38,PC,-0.008651,0.03,-0.013,37.868807,38.037,37.5,00778610,00778610
5891377,2019-07-28,HIVE-DEFUNCT,007786106,2428,2,2019,2019-07-30T18:29:38-04:00,154379,HIVE,007786106,...,18:29:38,PC,-0.008651,0.03,-0.013,37.868807,38.037,37.5,00778610,00778610
5891378,2019-07-29,HIVE-DEFUNCT,007786106,2428,2,2019,2019-07-30T18:29:38-04:00,154379,HIVE,007786106,...,18:29:38,PC,-0.004984,0.03,-0.013,37.802369,38.037,37.5,00778610,00778610


In [18]:
"""
Get release ID - cticker pair and use that API??? 

"""
# Setup and login keys for API 

url = "http://api.estimize.com" 
headers = {"X-Estimize-Key" : "ca2bed82074413cd06d2711f", 
           "content-type" : "application/json"}

def get_release_id(ticker):
    """
    Overview: returns a list of dictionaries of the past financial releases for the speficied company (by ticker)
    Required params: None
    Optional params: None
    Response: 
	-fiscal_year: The fiscal year for the release
	-fiscal_quarter: The fiscal quarter for the release
	-release_date: The date of the release
	-eps: The earnings per share for the specified fiscal quarter
	-revenue: The revenue for the speified fiscal quarter
	-wallstreet_eps_estimate: The estimated EPS from Wall Street
	-wallstreet_revenue_estimate: The estimated revenue from Wall Street
	-consensus_eps_estimate: The average estimated EPS by the Estimize community 
	-consensus_revenue_estimate: The average estimated revenue by the Estimize community
    """
    function_url = url + "/companies/%s/releases" % ticker
    req = requests.get(function_url, headers=headers)
    if req.status_code != 200 : return None
    return pd.DataFrame(req.json())

# Save information (fiscal year-quarter, final estimates, id and release date)
list_release_ids = []

# Just print out unique tickers 
unique_ctickers = cons2['cticker'].drop_duplicates()
unique_ctickers

0           MLM
4747        TXI
4765         CX
5819        VMC
10859       EXP
           ... 
6973317    ABCL
6973318    PLBY
6973331     ZIM
6973354    CRON
6973359    NVEI
Name: cticker, Length: 3051, dtype: object

In [19]:
# Get release id's for unique tickers from above 
for i in range(len(unique_ctickers)):
    temp2 = pd.DataFrame(get_release_id(unique_ctickers.iloc[i]))
    temp2['cticker'] = unique_ctickers.iloc[i]
    print(unique_ctickers.iloc[i])
    list_release_ids.append(temp2)
    
# Join together in data frame 
tics_event_ids = pd.concat(list_release_ids)
tics_event_ids

MLM
TXI
CX
VMC
EXP
HW
SHLM
CSTE
AGU
APD
ARG
ALB
HUN
IPHS
IOSP
IFF
IPI
KOP
KRA
LXU
LYB
MTX
MON
NEU
RPM
SIAL
SXT
KWR
ROC
AVNT
POT
LIN
PPG
AVD
AMRS
ASH
OLN
DOW-DEFUNCT
MOS
SMG
SHW
VAL-DEFUNCT
TG
GRA
WLK
ZEP
BCPC
CBT
CCC-DEFUNCT
CE
CF
SQM
CHMT
AXLL
FUL
FMC
ECL
CYT
DD-DEFUNCT
SCL
EMN
TROX
OCIP
KRO
FTK
UAN
RNF
FF
IP
LPX
KS
WPP
SWM
GLT
CLW
UFS
NP
BCC
CCK
MYE
MWV
SON
SEE
SLGN
RKT-DEFUNCT
ATR
OI
PNG
PKG
AVY
AMCR
BLL
GPK
GEF
COST
BERY
CVS
SPTN
SVU
SUSS
SYY
SWY
PSMT
TFM
ANDE
KR
UNFI
WMT
WFM
CASY
HNZ
RAD
SFM
NGVC
IMKTA
WBA
CAG
INGR
HRL
JJSF
K
KHC
LANC
MKC
MJN
SAFM
BDBD
SFD
LNCE
PPC
ADM
HAIN
SJM
THS
TSN
BGS
BG
CALM
CVGW
CPB
CQB
GMCR
HSY
FLO
FDP
GIS
DAR
DMND
DFODQ
DOLE
MDLZ
WWAV
POST
JMBA
PF
FRPT
KMB
CL
NUS
SPB
PG
PBH
USNA
WDFC
AVP
CENT
CHD
CLX
HLF
RDEN
EL
ENR
HRG
IPAR
STZ
TAP
PEP
KO
SAM
MNST
CCEP
KDP
MO
LO
RAI
PM
UVV
VGR
COTY
COLM
CROX
HOTT
ICON
KATE
LUX
MFB
MOV
NKE
SHOO
SKX
PVH
ZQKSQ
PERY
RL
OXM
JNY
TRLG
UAA
VFC
VRA
WWW
BODY
CRI
TPR
GIL
HBI
FOSL
GIII
DECK
CPRI
LULU
TUMI
VNCE
THO
TSLA
HOG
F
GM
WGO


FSLR
FORM
FSL
ENTG
ENTR
EZCH
FCS
CY
DIOD
SPWR
SUNEQ
GTATQ
AMBA
PDFS
QRVO
ACN
RAMP
BFH
DOX
CTSH
CSC
CVG
CSGS
ONE
PACT
IGTE
INXN
IBM
ISS
JKHY
LPS
MANT
MA
MMS
MGI
NSR
SYKE
SYNT
SAI
SAPE
SREV
PAYX
TTEC
ADP
TDC
TSS
UIS
PAY
VRTU
V
WU
BAH
BR
CACI
CATM
GPN
HPY
NESC
FLT
FISV
FORR
IT
G
EEFT
EXLS
FIS
DST
EPAM
WP
WEX
EVTC
LDOS
SAIC
ACIW
SABR
ATVI
ADBE
ALLT
ADVS
CVLT
CNQR
INFA
ININ
INTU
MANH
MENT
MCRS
MSFT
MSTR
TYPE
UEPS
NTCT
N
NUAN
FIRE
SSNC
NLOK
SNCR
SNPS
TTWO
ROVI
CRM
SWI
SLH
PRGS
PRO
QLIK
RHT
RP
PEGA
ANSS
ASIA
AZPN
ADSK
ORCL
TIBX
TYL
ULTI
VRNT
VMW
BLKB
EPAY
BSFT
CA
CDNS
CHKP
CTXS
FTNT
EBIX
EA
EPIQ
FDS
FICO
ZNGA
JIVE
GWRE
IMPV
TNGO
ET - Defunct
WDAY
SPLK
NOW
TIVO
BLOX
MNDT
PFPT
QLYS
CVT
ELLI
KING
DATA
ZEN
PTC
CUDA
SSNI
MESG
ENOC
TUBE
CYBR
HUBS
CCI
LEAP
NTLS
S
SBAC
AMT
TNAV
TKC
TDS
USM
SPOK
VEON
BCE-DEFUNCT
TMUS
RNG
CCOI
CNSL
IRDM
LVLT
T
PGI
VZ
VG
LUMN
CBB
FTR
DGII
EGHT
ZAYO
AEE
LNT
ED
MDU
NI
NWE
SCG
SRE
PEG
PCG
TE
VVC
WEC
AVA
BKH
CNP
CMS
D
DTE
ORA
NRG
AES
CPN
DYN
PEGI
VSLR
ALE
IDA
ITC
NEE
NU
SO


,fiscal_year,fiscal_quarter,eps,revenue,consensus_eps_estimate,consensus_revenue_estimate,wallstreet_revenue_estimate,wallstreet_eps_estimate,consensus_weighted_revenue_estimate,consensus_weighted_eps_estimate,release_date,id,cticker
0,2003.0,2.0,0.81,471.90,NaN,NaN,460.98,0.725,NaN,NaN,2003-06-01T04:00:00-04:00,4f066aefc6efa653ac00be0b,MLM
1,2005.0,3.0,1.47,566.70,NaN,NaN,581.96,1.45,NaN,NaN,2005-09-01T04:00:00-04:00,4f066aefc6efa653ac00be14,MLM
2,2005.0,4.0,1.04,503.80,NaN,NaN,467.85,0.9,NaN,NaN,2005-12-01T05:00:00-05:00,4f066aefc6efa653ac00be15,MLM
3,2009.0,4.0,-0.07,374.69,NaN,NaN,383.01,0.33,NaN,NaN,2009-12-01T00:00:00-05:00,4f066af0c6efa653ac00be25,MLM
4,2012.0,3.0,1.36,593.90,NaN,NaN,519.883,1.26,NaN,NaN,2012-11-06T06:00:00-05:00,4f066af0c6efa653ac00be30,MLM
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13,2020.0,2.0,0.18,83.33,NaN,NaN,NaN,NaN,NaN,NaN,2020-08-10T06:00:00-04:00,211635,NVEI
14,2020.0,1.0,0.11,83.24,NaN,NaN,NaN,NaN,NaN,NaN,2020-05-10T06:00:00-04:00,211636,NVEI
15,2019.0,4.0,0.06,79.33,NaN,NaN,NaN,NaN,NaN,NaN,2020-03-10T06:00:00-04:00,211637,NVEI
16,2019.0,3.0,0.03,70.75,NaN,NaN,NaN,NaN,NaN,NaN,2019-11-11T06:00:00-05:00,211638,NVEI


In [20]:
"""
SQL step to join the estimize_id (unique identifier) to connect it with 
the API database of ctickers. 

Must match on the release_id (unique to each earnings announcement date) 
and to the current ticker. 

"""
con = sqlite3.connect("estimize.db")
cons2.to_sql("cons2", con, index=False, if_exists='replace')
tics_event_ids.to_sql("tics_event_ids", con, index=False, if_exists='replace')

sql = """SELECT a.*, b.release_date as ea_datetime, b.cticker as cticker_api
FROM cons2 as a, tics_event_ids as b
where (a.cticker=b.cticker)
AND (a.release_id = b.id)
order  by a.exp_ea_date, a.hticker, a.hcusip9;"""

estimize_joined = pd.read_sql_query(sql, con)
estimize_joined

,date,cticker,ccusip9,estimize_id,fqtr,fyear,exp_ea_datetime,release_id,hticker,hcusip9,...,eps_wgt,eps_act,eps_ws,sales_wgt,sales_act,sales_ws,hcusip,ccusip,ea_datetime,cticker_api
0,2015-05-10,SHAK,819047101,2309,1,2015,2015-05-13T16:00:00-04:00,122400,SHAK,819047101,...,-0.018004,0.04,-0.033000,32.816546,37.808,33.444000,81904710,81904710,2015-05-13T16:00:00-04:00,SHAK
1,2015-05-11,SHAK,819047101,2309,1,2015,2015-05-13T16:00:00-04:00,122400,SHAK,819047101,...,-0.013079,0.04,-0.033000,34.169019,37.808,33.444000,81904710,81904710,2015-05-13T16:00:00-04:00,SHAK
2,2015-05-12,SHAK,819047101,2309,1,2015,2015-05-13T16:00:00-04:00,122400,SHAK,819047101,...,-0.009908,0.04,-0.033000,34.256643,37.808,33.444000,81904710,81904710,2015-05-13T16:00:00-04:00,SHAK
3,2015-05-13,SHAK,819047101,2309,1,2015,2015-05-13T16:00:00-04:00,122400,SHAK,819047101,...,-0.013597,0.04,-0.033000,34.410476,37.808,33.444000,81904710,81904710,2015-05-13T16:00:00-04:00,SHAK
4,2015-05-15,ETSY,29786A106,1913,1,2015,2015-05-19T16:00:00-04:00,123047,ETSY,29786A106,...,-0.041667,-0.12,NaN,69.403782,58.543,NaN,29786A10,29786A10,2015-05-19T16:00:00-04:00,ETSY
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6169648,2022-03-27,INTC,458140100,1950,4,2023,2024-01-25T16:00:00-05:00,209456,INTC,458140100,...,NaN,NaN,0.990347,20960.906623,NaN,20232.351095,45814010,45814010,2024-01-25T16:00:00-05:00,INTC
6169649,2022-03-28,INTC,458140100,1950,4,2023,2024-01-25T16:00:00-05:00,209456,INTC,458140100,...,NaN,NaN,0.990347,20960.906623,NaN,20232.351095,45814010,45814010,2024-01-25T16:00:00-05:00,INTC
6169650,2022-03-29,INTC,458140100,1950,4,2023,2024-01-25T16:00:00-05:00,209456,INTC,458140100,...,NaN,NaN,0.990347,20960.906623,NaN,20232.351095,45814010,45814010,2024-01-25T16:00:00-05:00,INTC
6169651,2022-03-30,INTC,458140100,1950,4,2023,2024-01-25T16:00:00-05:00,209456,INTC,458140100,...,NaN,NaN,0.990347,20960.906623,NaN,20232.351095,45814010,45814010,2024-01-25T16:00:00-05:00,INTC


In [21]:
"""
Timestamp column - EA Datetime pull apart into separate columns after 
SQL merging. 

"""

# Get rid of the T delimiter in the timestamp initially 
estimize_joined['temp'] = estimize_joined['ea_datetime'].str.replace("T", " ", regex=True)

# Filter out the offset (will be automatically handled and detected by tz_localize)
estimize_joined['temp2'] = estimize_joined['temp'].str[0:19]

# Get the Eastern timestamp from converted to datetime column 
estimize_joined['temp2'] = pd.to_datetime(estimize_joined['temp2'])
estimize_joined['ea_time'] = pd.to_datetime(estimize_joined['ea_datetime'].str[11:19]).dt.time

# Last but not least get the ea_date extracted
estimize_joined['ea_date'] = estimize_joined['temp2'].dt.strftime('%Y-%m-%d')

# Don't need temp columns anymore
estimize_joined = estimize_joined[['date','cticker','ccusip9','estimize_id','fqtr','fyear',
                                   'exp_ea_datetime','eps_wgt','eps_ws',
                                   'eps_act','release_id','hticker', 'ea_datetime', 
                                   "ea_date", "ea_time", 'hcusip9','root','suffix',
                                   'exp_ea_date','exp_ea_time', 'hcusip','ccusip',
                                   "sales_wgt", "sales_ws", "sales_act", 'exp_ea_tod'
                                  ]]

estimize_joined

,date,cticker,ccusip9,estimize_id,fqtr,fyear,exp_ea_datetime,eps_wgt,eps_ws,eps_act,...,root,suffix,exp_ea_date,exp_ea_time,hcusip,ccusip,sales_wgt,sales_ws,sales_act,exp_ea_tod
0,2015-05-10,SHAK,819047101,2309,1,2015,2015-05-13T16:00:00-04:00,-0.018004,-0.033000,0.04,...,SHAK,,2015-05-13,16:00:00.000000,81904710,81904710,32.816546,33.444000,37.808,PC
1,2015-05-11,SHAK,819047101,2309,1,2015,2015-05-13T16:00:00-04:00,-0.013079,-0.033000,0.04,...,SHAK,,2015-05-13,16:00:00.000000,81904710,81904710,34.169019,33.444000,37.808,PC
2,2015-05-12,SHAK,819047101,2309,1,2015,2015-05-13T16:00:00-04:00,-0.009908,-0.033000,0.04,...,SHAK,,2015-05-13,16:00:00.000000,81904710,81904710,34.256643,33.444000,37.808,PC
3,2015-05-13,SHAK,819047101,2309,1,2015,2015-05-13T16:00:00-04:00,-0.013597,-0.033000,0.04,...,SHAK,,2015-05-13,16:00:00.000000,81904710,81904710,34.410476,33.444000,37.808,PC
4,2015-05-15,ETSY,29786A106,1913,1,2015,2015-05-19T16:00:00-04:00,-0.041667,NaN,-0.12,...,ETSY,,2015-05-19,16:00:00.000000,29786A10,29786A10,69.403782,NaN,58.543,PC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6169648,2022-03-27,INTC,458140100,1950,4,2023,2024-01-25T16:00:00-05:00,NaN,0.990347,NaN,...,INTC,,2024-01-25,16:00:00.000000,45814010,45814010,20960.906623,20232.351095,NaN,PC
6169649,2022-03-28,INTC,458140100,1950,4,2023,2024-01-25T16:00:00-05:00,NaN,0.990347,NaN,...,INTC,,2024-01-25,16:00:00.000000,45814010,45814010,20960.906623,20232.351095,NaN,PC
6169650,2022-03-29,INTC,458140100,1950,4,2023,2024-01-25T16:00:00-05:00,NaN,0.990347,NaN,...,INTC,,2024-01-25,16:00:00.000000,45814010,45814010,20960.906623,20232.351095,NaN,PC
6169651,2022-03-30,INTC,458140100,1950,4,2023,2024-01-25T16:00:00-05:00,NaN,0.990347,NaN,...,INTC,,2024-01-25,16:00:00.000000,45814010,45814010,20960.906623,20232.351095,NaN,PC


In [22]:
"""
Sort the table, check ordering, and export out. 

"""
# Sort and get rid of temporary column for creating trading hours flag
cons2 = cons2.sort_values(by=['hticker', "exp_ea_date"])

# Save as parquet
cons2.to_parquet(onedrive + out_pth + "estimize_cons_exp_eadates.parquet")

# Check only on Apple ticker
cons2[cons2['hticker']=="AAPL"]

# Sort and get rid of temporary column for creating trading hours flag
estimize_joined = estimize_joined.sort_values(by=['hticker', "ea_date"])

# Save as parquet
estimize_joined.to_parquet(onedrive + out_pth + "estimize_cons_eadates.parquet")

# Check only on Apple ticker
estimize_joined[estimize_joined['hticker']=="AAPL"]

,date,cticker,ccusip9,estimize_id,fqtr,fyear,exp_ea_datetime,eps_wgt,eps_ws,eps_act,...,root,suffix,exp_ea_date,exp_ea_time,hcusip,ccusip,sales_wgt,sales_ws,sales_act,exp_ea_tod
169436,2016-01-11,AAPL,037833100,1759,2,2017,2017-05-02T16:00:00-04:00,0.602520,0.635000,0.525,...,AAPL,,2017-05-02,16:00:00.000000,03783310,03783310,59250.329919,62095.000000,52896.0,PC
169437,2016-01-12,AAPL,037833100,1759,2,2017,2017-05-02T16:00:00-04:00,0.602520,0.635000,0.525,...,AAPL,,2017-05-02,16:00:00.000000,03783310,03783310,59250.329919,60080.750000,52896.0,PC
169438,2016-01-13,AAPL,037833100,1759,2,2017,2017-05-02T16:00:00-04:00,0.602520,0.623250,0.525,...,AAPL,,2017-05-02,16:00:00.000000,03783310,03783310,59250.329919,60080.750000,52896.0,PC
169439,2016-01-14,AAPL,037833100,1759,2,2017,2017-05-02T16:00:00-04:00,0.602520,0.623250,0.525,...,AAPL,,2017-05-02,16:00:00.000000,03783310,03783310,59250.329919,60080.750000,52896.0,PC
169440,2016-01-15,AAPL,037833100,1759,2,2017,2017-05-02T16:00:00-04:00,0.602520,0.623250,0.525,...,AAPL,,2017-05-02,16:00:00.000000,03783310,03783310,59250.329919,60080.750000,52896.0,PC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6143369,2022-03-27,AAPL,037833100,1759,1,2023,2023-01-31T16:00:00-05:00,2.253826,2.226671,NaN,...,AAPL,,2023-01-31,16:00:00.000000,03783310,03783310,133434.783825,133942.360388,NaN,PC
6143370,2022-03-28,AAPL,037833100,1759,1,2023,2023-01-31T16:00:00-05:00,2.253826,2.226671,NaN,...,AAPL,,2023-01-31,16:00:00.000000,03783310,03783310,133434.783825,133942.360388,NaN,PC
6143371,2022-03-29,AAPL,037833100,1759,1,2023,2023-01-31T16:00:00-05:00,2.266998,2.226671,NaN,...,AAPL,,2023-01-31,16:00:00.000000,03783310,03783310,134696.101980,133942.360388,NaN,PC
6143372,2022-03-30,AAPL,037833100,1759,1,2023,2023-01-31T16:00:00-05:00,2.266998,2.226671,NaN,...,AAPL,,2023-01-31,16:00:00.000000,03783310,03783310,134696.101980,133942.360388,NaN,PC


In [ ]:
"""
(NOTE: Code concludes above!)
The rest of these cells are just checks and stuff to be run separately 

"""
estimize_joined[estimize_joined['hticker'].str.contains("-")]

In [ ]:
estimize_joined[estimize_joined['hticker'].str.contains("DEFUNCT", case=False)]

In [ ]:
"""
Check against GE and Wall Street Horizon data 

"""
cons2[cons2['hticker']=="GE"]

In [ ]:
# Check against WSH
blah = pd.read_parquet(
    "C://Users/clj585/OneDrive - Northwestern University/data_feeds/WSH/data/wsh_earn_eadates.parquet")
#blah2 = blah[blah['hticker']=='GE'].sort_values(by='ea_date')
blah[blah['fyear']>=2010]

In [ ]:
estimize_joined[['cticker', 'estimize_id']]

In [ ]:
estimize_joined[['cticker', 'estimize_id']].drop_duplicates()

In [ ]:
estimize_joined[['cticker', 'estimize_id']].drop_duplicates(['estimize_id'])

In [ ]:
temp = estimize_joined[['cticker', 'estimize_id']].drop_duplicates()
temp[temp['cticker'].str.contains("DEFUNCT")]

In [ ]:
estimize_joined[estimize_joined['cticker'].str.contains("DEFUNCT")]

In [ ]:
estimize_joined[estimize_joined['cticker']=="DOW-DEFUNCT"]

In [ ]:
pd.DataFrame(get_release_id("DOW-DEFUNCT"))

In [ ]:
len(cons2[['cticker', 'estimize_id']].drop_duplicates())

In [ ]:
len(estimize_joined[['cticker', 'estimize_id']].drop_duplicates())

In [ ]:
temp1 = cons2[['cticker', 'estimize_id']].drop_duplicates()
temp2 = estimize_joined['cticker'].drop_duplicates()


con2 = sqlite3.connect("estimize2.db")
temp1.to_sql("temp1", con2, index=False, if_exists='replace')
temp2.to_sql("temp2", con2, index=False, if_exists='replace')

sql2 = """SELECT a.*, b.cticker as cticker_b
FROM temp1 as a LEFT JOIN temp2 as b
on (a.cticker=b.cticker)
order  by a.cticker;"""

check = pd.read_sql_query(sql2, con2)
check

In [ ]:
check[check['cticker_b'].isnull()]

In [ ]:
get_release_id("ACHI")

In [ ]:
min(cons2['date'])